In [64]:
import os
import torch
import torch.nn as nn
import torchvision
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset


In [59]:
class ZeroShotDefectClassifier(nn.Module):
    def __init__(self, num_attributes, num_classes):
        super(ZeroShotDefectClassifier, self).__init__()
        self.features = nn.Sequential(
            # feature extraction layers
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((4, 4))  # Adjust output size
        self.classifier = nn.Linear(256 * 4 * 4, num_attributes)  # Adjust input size
        self.fc = nn.Linear(num_attributes, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        x = self.fc(x)
        return x


In [60]:
# Initialize the model
num_attributes = 10  
num_classes = 2  # Number of classes (good or broken)
model = ZeroShotDefectClassifier(num_attributes, num_classes)

# Loss function
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [61]:
# Dataset and dataloader for training data
batch_size = 32
data_dir = 'D:/MvTec/mvtec_anomaly_detection'
class_name = 'bottle'
class_dir = os.path.join(data_dir, class_name)
train_dir = os.path.join(class_dir, 'Train')
product1_train_dataset = ImageFolder(train_dir, transform=transform)
product1_train_loader = DataLoader(product1_train_dataset, batch_size=batch_size, shuffle=True)

class_name = 'carpet'
class_dir = os.path.join(data_dir, class_name)
train_dir = os.path.join(class_dir, 'Train')
product2_train_dataset = ImageFolder(train_dir, transform=transform)
product2_train_loader = DataLoader(product2_train_dataset, batch_size=batch_size, shuffle=True)


In [62]:
# Combine the data loaders 
train_datasets = [product1_train_dataset, product2_train_dataset]  # Add more datasets as needed
combined_train_dataset = ConcatDataset(train_datasets)
combined_train_loader = DataLoader(combined_train_dataset, batch_size=batch_size, shuffle=True)


In [65]:
# Define the directory paths for other classes
train_dirs = [
    os.path.join(dataset_root, "cable", "train"),
    os.path.join(dataset_root, "carpet", "train"),
    os.path.join(dataset_root, "capsule", "train"),
    os.path.join(dataset_root, "hazelnut", "train")
]

# Initialize the model for each class
models = []
for _ in range(len(classes)):
    model = ZeroShotDefectClassifier(num_attributes, num_classes)
    model.to(device)
    models.append(model)

# Training loop for each class
for class_index, train_dir in enumerate(train_dirs):
    train_dataset = torchvision.datasets.ImageFolder(train_dir, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    model = models[class_index]
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_loader)
        print(f"Class: {classes[class_index]}, Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

Class: bottle, Epoch [1/10], Loss: 0.6726
Class: bottle, Epoch [2/10], Loss: 0.0464
Class: bottle, Epoch [3/10], Loss: 0.0000
Class: bottle, Epoch [4/10], Loss: 0.0000
Class: bottle, Epoch [5/10], Loss: 0.0000
Class: bottle, Epoch [6/10], Loss: 0.0000
Class: bottle, Epoch [7/10], Loss: 0.0000
Class: bottle, Epoch [8/10], Loss: 0.0000
Class: bottle, Epoch [9/10], Loss: 0.0000
Class: bottle, Epoch [10/10], Loss: 0.0000
Class: cable, Epoch [1/10], Loss: 0.5829
Class: cable, Epoch [2/10], Loss: 0.1080
Class: cable, Epoch [3/10], Loss: 0.0000
Class: cable, Epoch [4/10], Loss: 0.0000
Class: cable, Epoch [5/10], Loss: 0.0000
Class: cable, Epoch [6/10], Loss: 0.0000
Class: cable, Epoch [7/10], Loss: 0.0000
Class: cable, Epoch [8/10], Loss: 0.0000
Class: cable, Epoch [9/10], Loss: 0.0000
Class: cable, Epoch [10/10], Loss: 0.0000
Class: carpet, Epoch [1/10], Loss: 0.4640
Class: carpet, Epoch [2/10], Loss: 0.0836
Class: carpet, Epoch [3/10], Loss: 0.0002
Class: carpet, Epoch [4/10], Loss: 0.0000


In [70]:
batch_size = 32
class_labels = ['bottle', 'carpet', 'capsule', 'hazelnut']
defect_labels = {
    'bottle': ['broken_large', 'broken_small', 'contamination'],
    'carpet': ['color', 'cut', 'hole','metal_contamination','thread'],
    'capsule': ['crack', 'poke', 'scratch','faulty_imprint','squeeze' ],
    'hazelnut': ['cut', 'crack', 'hole','print']
}

# Initialize variables for counting correct predictions
correct_predictions_by_class = {class_label: {defect_label: 0 for defect_label in defect_labels[class_label]} for class_label in class_labels}
total_samples_by_class = {class_label: {defect_label: 0 for defect_label in defect_labels[class_label]} for class_label in class_labels}

# Iterate over the test data and labels for each class
for class_label in class_labels:
    test_dir = os.path.join(data_dir, class_label, "test")
    test_dataset = ImageFolder(test_dir, transform=transform)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=False)
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        # Count correct predictions for defect classes
        for defect_label in defect_labels[class_label]:
            defect_samples = (labels == test_dataset.class_to_idx[defect_label]).sum().item()
            correct_predictions_by_class[class_label][defect_label] += ((predicted == labels) & (labels == test_dataset.class_to_idx[defect_label])).sum().item()
            total_samples_by_class[class_label][defect_label] += defect_samples

# Calculate precision for defect classes
precision_by_class = {}
for class_label in class_labels:
    precision_by_class[class_label] = {}
    for defect_label in defect_labels[class_label]:
        precision_by_class[class_label][defect_label] = correct_predictions_by_class[class_label][defect_label] / total_samples_by_class[class_label][defect_label]

# Calculate overall accuracy and precision
total_samples = sum([sum(total_samples_by_class[class_label].values()) for class_label in class_labels])
correct_predictions = sum([sum(correct_predictions_by_class[class_label].values()) for class_label in class_labels])
accuracy = correct_predictions / total_samples

# Print the results
print("Accuracy: {:.4f}".format(accuracy))
print("Precision by Class:")
for class_label in class_labels:
    print(f" - {class_label}:")
    for defect_label in defect_labels[class_label]:
        print(f"   - {defect_label}: {precision_by_class[class_label][defect_label]:.4f}")


Accuracy: 0.2417
Precision by Class:
 - bottle:
   - broken_large: 1.0000
   - broken_small: 0.0000
   - contamination: 0.0000
 - carpet:
   - color: 1.0000
   - cut: 0.0000
   - hole: 0.0000
   - metal_contamination: 0.0000
   - thread: 0.0000
 - capsule:
   - crack: 1.0000
   - poke: 0.0000
   - scratch: 0.0000
   - faulty_imprint: 0.0000
   - squeeze: 0.0000
 - hazelnut:
   - cut: 0.0000
   - crack: 1.0000
   - hole: 0.0000
   - print: 0.0000
